In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatib

In [2]:
# Importing Relevant Packages
from ortools.sat.python import cp_model
import pandas as pd
import time
from dataclasses import dataclass
from typing import List
import re
import csv
from collections import defaultdict
import time
import matplotlib.pyplot as plt
import numpy as np
from statistics import stdev
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import random
from itertools import product
import json
from google.colab import files
from io import BytesIO

In [3]:
@dataclass
class Patient: # Define a data class for Patient
    id: int # Unique identifier for the patient
    name: str # Name of the patient
    age: int # Age of the patient
    gender: str # Gender of the patient
    admission_day: int # Day the patient is admitted
    release_day: int # Day the patient is released
    spec_id: int # Specialism ID required by the patient
    spec_days: int # Number of days the specialism is required
    prefered_capacity: int # Preferred room capacity for the patient
    needs: List[int] # List of equipment the patient needs (represented as integers)
    prefers: List[int] # List of equipment the patient prefers (represented as integers)

@dataclass
class Room: # Define a data class for Room
    id: int # Unique identifier for the room
    room_number: int # Room number
    department: int # Department ID the room belongs to
    capacity: int # Maximum capacity of the room
    gender: str # Gender the room is designated for
    has: List[int] # List of equipment the room has (represented as integers)
    spec_ids: List[int] # List of specialism IDs supported by the room
    penalties: List[int] # List of penalties associated with specialisms in this room

In [4]:
uploaded = files.upload()

# Read the content of 'cleaned_rooms1.csv' from the uploaded files and load it into a pandas DataFrame
rooms_df    = pd.read_csv(BytesIO(uploaded['cleaned_rooms1.csv']))
# Read the content of 'cleaned_patients1.csv' from the uploaded files and load it into a pandas DataFrame
patients_df = pd.read_csv(BytesIO(uploaded['cleaned_patients1.csv']))

Saving cleaned_patients1.csv to cleaned_patients1.csv
Saving cleaned_rooms1.csv to cleaned_rooms1.csv


In [5]:
# Function to build hospital data from DataFrames
def build_hospital_data():
    # Create a list of Room objects from the rooms_df DataFrame
    rooms = [
        Room(
            id=i, # Assign a unique ID to each room
            room_number=int(row["RoomNumber"]), # Get room number
            department=int(row["DepartmentID"]), # Get department ID
            capacity=int(row["Capacity"]), # Get room capacity
            gender=row["Gender"], # Get gender designation
            has=[ # List of equipment the room has (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["Telemetry"]),
                int(row["Oxygen"]),
                int(row["Nitrogen"]),
                int(row["Television"])
            ],
            spec_ids=[ # List of specialism IDs supported by the room
                int(row["ReqSpecialism1"]),
                int(row["ReqSpecialism2"]),
                int(row["ReqSpecialism3"])
            ],
            penalties=[ # List of penalties associated with specialisms in this room
                int(row["PenaltySpecialism1"]),
                int(row["PenaltySpecialism2"]),
                int(row["PenaltySpecialism3"])
            ]
        )
        for i, row in rooms_df.iterrows() # Iterate through each row of the rooms_df DataFrame
    ]

    # Create a list of Patient objects from the patients_df DataFrame
    patients = [
        Patient(
            id=i, # Assign a unique ID to each patient
            name=row["Name"], # Get patient name
            age=int(row["Age"]), # Get patient age
            gender=row["Gender"], # Get patient gender
            admission_day=int(row["AdmissionDay"]), # Get admission day
            release_day=int(row["ReleaseDay"]), # Get release day
            spec_id=int(row["SpecialismID"]), # Get specialism ID
            spec_days=int(row["SpecialismDays"]), # Get number of days specialism is needed
            prefered_capacity=int(row["PreferredRoomCapacity"]), # Get preferred room capacity
            needs=[ # List of equipment the patient needs (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["NeedsTelemetry"]),
                int(row["NeedsOxygen"]),
                int(row["NeedsNitrogen"]),
                int(row["NeedsTV"])
            ],
            prefers=[ # List of equipment the patient prefers (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["PrefersTelemetry"]),
                int(row["PrefersOxygen"]),
                int(row["PrefersNitrogen"]),
                int(row["PrefersTV"])
            ]
        )
        for i, row in patients_df.iterrows() # Iterate through each row of the patients_df DataFrame
    ]

    # Return the lists of Room and Patient objects
    return rooms, patients

In [6]:
# Custom solution callback for printing the final hospital allocation
class HospitalAllocationPrinter(cp_model.CpSolverSolutionCallback):
    # Constructor to initialize the callback with necessary data
    def __init__(self, seats, patients, rooms):
        super().__init__() # Call the constructor of the parent class
        self.__seats = seats # Store the dictionary of boolean variables representing assignments
        self.__patients = patients # Store the list of patient objects
        self.__rooms = rooms # Store the list of room objects
        self.__solution_found = False # Flag to indicate if a solution has been found

    # Callback method called by the solver when an improving solution is found
    def on_solution_callback(self):
        self.__solution_found = True # Set the flag to True

    # Method to print the final solution after the solver finishes
    def print_final_solution(self, solver):
        print("\nFinal solution:") # Print a header for the final solution
        # Iterate through each room to display its assignments
        for room in self.__rooms:
            # Print room details
            print(f"\nRoom {room.id} (RoomNum: {room.room_number}, Dept: {room.department}, Capacity: {room.capacity}, Gender: {room.gender}):")
            assigned = False # Flag to check if any patient is assigned to this room
            # Iterate through each patient to see if they are assigned to the current room
            for patient in self.__patients:
                # Check if the boolean variable for this patient-room assignment is True in the final solution
                if solver.BooleanValue(self.__seats[(room.id, patient.id)]):
                    # Print patient details if assigned to this room
                    print(f"  - Patient {patient.name} (Age: {patient.age}, Gender: {patient.gender})")
                    assigned = True # Set flag to True as a patient is assigned
            # If no patient was assigned to this room, print "(empty)"
            if not assigned:
                print("  (empty)")

    # Method to check if a solution has been found
    def solution_found(self):
        return self.__solution_found

In [7]:
# Gini Coefficient is computed
def compute_gini(x):
    x = np.array(x) # Array with non-negative values
    if np.all(x == 0):  # If everyone has zero, then it is perfect equitablity
        return 0.0
    diff_sum = np.sum(np.abs(np.subtract.outer(x, x))) # Or else pairwise normalised absolute differences are computed
    return diff_sum / (2 * len(x) * np.sum(x))

# Jain's Index is computed
def compute_jain(x):
    x = np.array(x) # Array with non-negative values
    if np.sum(x) == 0:
        return 1.0 # If everyone has zero, then it is perfect equitablity
    return (np.sum(x)**2) / (len(x) * np.sum(x**2)) # Or else this is returned

In [8]:
# Function to calculate a compatibility penalty between a patient and a room
def compatibility_penalty(p, r, weights):
    pen = 0 # Initialize penalty to zero
    # Add penalty if room capacity is greater than the patient's preferred capacity
    if r.capacity > p.prefered_capacity:
        pen += weights['Wcap']
    # Add penalty if room gender is specified and doesn't match patient gender
    if r.gender in ['F', 'M'] and r.gender != p.gender:
        pen += weights['Wgen']
    # Add penalty if any preferred equipment is not available in the room
    if any(p.prefers[i] == 1 and r.has[i] == 0 for i in range(4)): # Assuming 4 equipment types
        pen += weights['Wpeq']
    # Add penalty if any needed equipment is not available in the room
    if any(p.needs[i] == 1 and r.has[i] == 0 for i in range(4)): # Assuming 4 equipment types
        pen += weights['Wneq']
    # Add penalty if patient age violates department age limits (assuming specific department IDs and age limits)
    if (r.department == 1 and p.age < 65) or (r.department == 4 and p.age > 16):
        pen += weights['Wage']
    # Add penalty based on specialism compatibility and penalty level
    if p.spec_id in r.spec_ids:
        idx = r.spec_ids.index(p.spec_id) # Find the index of the patient's specialism in the room's specialism list
        spec_penalty = r.penalties[idx] # Get the penalty value associated with that specialism in the room
        if spec_penalty == 2: # Minor specialism mismatch (penalty 2)
            pen += weights['Wspec_minor']
        elif spec_penalty >= 3: # Major specialism mismatch (penalty >= 3)
            pen += weights['Wspec_major']
    else: # Specialism not supported by the room
        pen += weights['Wspec_mismatch']
    return pen # Return the calculated total penalty

In [9]:
# Main function to solve the hospital allocation problem with various weighted constraints
def solve_hospital_allocation(
    Wcap=0, Wgen=0, Wpeq=0, Wneq=0, Wage=0, # Weights for capacity, gender, preferred equipment, needed equipment, age
    Wspec_minor=0, Wspec_major=0, Wspec_mismatch=0, # Weights for specialism penalties (minor, major, mismatch)
    Wfair_high=0, Wfair_med=0, Wfair_low=0, # Weights for fairness violations (high, medium, low thresholds)
    Wmove=0 # Weight for patient movement penalty
):

    # 1) Build hospital data (rooms and patients)
    rooms, patients = build_hospital_data() # Loads room and patient data

    # Store weights in a dictionary for easy access
    weights = {
        'Wcap': Wcap, 'Wgen': Wgen, 'Wpeq': Wpeq, 'Wneq': Wneq, 'Wage': Wage,
        'Wspec_minor': Wspec_minor, 'Wspec_major': Wspec_major, 'Wspec_mismatch': Wspec_mismatch, 'Wfair_high': Wfair_high,
        'Wfair_med': Wfair_med, 'Wfair_low': Wfair_low, 'Wmove': Wmove
    }
    random.seed(42) # Set random seed for reproducibility
    np.random.seed(42) # Set numpy random seed for reproducibility

    # Metrics bookkeeping - Initialize variables to track performance and fairness
    total_duration = 0.0 # Total time taken to solve all days
    historical_penalties = {p.id: 0 for p in patients} # Dictionary to store cumulative penalties for each patient
    previous_assignments = {} # Dictionary to store the previous room assignment for each patient
    daily_assignment_penalties = {} # Dictionary to store penalties for the current day's assignments (not explicitly used here, but initialized)

    # 2) Horizon: days 0 … last admission day - Determine the range of days to consider
    last_admission_day = max(p.admission_day for p in patients) # Find the latest admission day among all patients
    all_days = range(last_admission_day + 1) # Create a range of all days from 0 up to the last admission day

    all_assignments = {} # Dictionary to store assignment results for each day
    results = [] # List to store summary results for each day

    # Solve day by day to allow threshold recalculation - Iterate through each day
    for current_day in all_days:
        print(f"Solving day {current_day}...") # Print the current day being solved
        # Get penalties from the previous day (not directly used for thresholds here, but available)
        prev_penalties = daily_assignment_penalties.get(current_day - 1, {})
        # Find the maximum penalty from the previous day (used for FCFS)
        max_prev_penalty = max(prev_penalties.values(), default=None)

        # Simulate possible patient schedule changes ON THIS DAY - Introduce uncertainty in patient schedules
        for p in patients:

            # Delay admission (only if admission is today)
            if p.admission_day == current_day:
                if random.random() < 0.3: # 30% chance of delay
                    delay = random.randint(1, 2) # Delay by 1 or 2 days
                    duration = p.release_day - p.admission_day # Calculate original stay duration
                    p.admission_day += delay # Update admission day
                    p.release_day = p.admission_day + duration # Update release day

                    if not hasattr(p, "arrival_delays"): # Initialize list if it doesn't exist
                        p.arrival_delays = []
                    # Log the delay event
                    p.arrival_delays.append({
                        "day": current_day,
                        "delay": delay,
                        "new_admission": p.admission_day,
                        "new_release": p.release_day,
                    })

            # Early admission (only if admission is today)
            if p.admission_day == current_day:
                if random.random() < 0.3: # 30% chance of early admission
                    advance = random.randint(1, 2) # Advance by 1 or 2 days
                    duration = p.release_day - p.admission_day # Calculate original stay duration
                    p.admission_day = max(0, p.admission_day - advance) # Update admission day (ensure it's not negative)
                    p.release_day = p.admission_day + duration # Update release day

                    if not hasattr(p, "early_arrivals"): # Initialize list if it doesn't exist
                        p.early_arrivals = []
                    # Log the early arrival event
                    p.early_arrivals.append({
                        "day": current_day,
                        "advance": advance,
                        "new_admission": p.admission_day,
                        "new_release": p.release_day,
                    })

            # Late release (only if release is today)
            if p.release_day == current_day:
                if random.random() < 0.3: # 30% chance of late release
                    extension = random.randint(1, 2) # Extend by 1 or 2 days
                    p.release_day += extension # Update release day

                    if not hasattr(p, "late_releases"): # Initialize list if it doesn't exist
                        p.late_releases = []
                    # Log the late release event
                    p.late_releases.append({
                        "day": current_day,
                        "extension": extension,
                        "new_release": p.release_day,
                    })

            # Early release (only if release is today)
            if p.release_day == current_day:
                if random.random() < 0.3: # 30% chance of early release
                    reduction = random.randint(1, 2) # Reduce by 1 or 2 days
                    new_release = max(p.admission_day + 1, p.release_day - reduction) # Calculate new release day (ensure stay is at least 1 day)
                    if new_release < p.release_day: # Only log if release day actually changed
                        if not hasattr(p, "early_releases"): # Initialize list if it doesn't exist
                            p.early_releases = []
                        # Log the early release event
                        p.early_releases.append({
                            "day": current_day,
                            "early_by": p.release_day - new_release,
                            "new_release": new_release,
                        })
                        p.release_day = new_release # Update release day

        # Identify patients present on the current day after potential schedule changes
        present_patients = [p for p in patients if p.admission_day <= current_day < p.release_day]
        if not present_patients: # If no patients are present, skip to the next day
            continue

        # Calculate fairness thresholds based on historical penalties or base penalties
        if current_day > 0:
            # For days after the first, calculate fairness scores considering historical penalties
            all_fairness_scores = []
            for p in present_patients:
                for r in rooms:
                    # Calculate base penalty using the compatibility_penalty function with weights
                    base_pen = compatibility_penalty(p, r, weights)
                    # Apply bias based on historical penalty
                    bias = 1 + historical_penalties[p.id] / max(1, current_day + 1) # Avoid division by zero
                    fairness_score = base_pen * bias
                    all_fairness_scores.append(fairness_score)

            # Calculate percentiles for thresholding
            scores_array = np.array(all_fairness_scores)
            threshold_low = np.percentile(scores_array, 25) # 25th percentile for low threshold
            threshold_high = np.percentile(scores_array, 67) # 67th percentile for high threshold

            print(f"Day {current_day} thresholds - Low: {threshold_low:.2f}, High: {threshold_high:.2f}")
        else:
            # For the first day, calculate thresholds based on base penalties only
            all_base_scores = []
            for p in present_patients:
                for r in rooms:
                    # Calculate base penalty using the compatibility_penalty function with weights
                    base_pen = compatibility_penalty(p, r, weights)
                    all_base_scores.append(base_pen)

            # Calculate percentiles for thresholding
            scores_array = np.array(all_base_scores)
            threshold_low = np.percentile(scores_array, 25) # 25th percentile for low threshold
            threshold_high = np.percentile(scores_array, 67) # 67th percentile for high threshold

            print(f"Day {current_day} (initial) thresholds - Low: {threshold_low:.2f}, High: {threshold_high:.2f}")

        # 3) CP-SAT Model Definition
        model = cp_model.CpModel() # Create a new CP-SAT model
        seats = {} # Dictionary to store boolean variables representing patient-room assignments
        # Create a boolean variable for each possible patient-room assignment on the current day
        for r in rooms:
            for p in present_patients:
                seats[(r.id, p.id)] = model.NewBoolVar(f"s_r{r.id}_p{p.id}_d{current_day}")

        # Constraints
        # Each patient must be assigned to exactly one room
        for p in present_patients:
            model.Add(sum(seats[(r.id, p.id)] for r in rooms) == 1)

        # Each room's capacity must not be exceeded
        for r in rooms:
            model.Add(sum(seats[(r.id, p.id)] for p in present_patients) <= r.capacity)

        # Equipment requirement constraint (Hard Constraint)
        # Determine the maximum number of equipment types to consider
        num_equipment_types = min(
            max(len(r.has) for r in rooms),
            max(len(p.needs) for p in present_patients)
        )

        # Enforce equipment need constraint for each equipment type
        for i in range(num_equipment_types):
            # Identify rooms that have the current equipment type
            rooms_with_equipment = [r for r in rooms if len(r.has) > i and r.has[i] == 1]
            total_available = len(rooms_with_equipment) # Count of rooms with the equipment

            # Identify patients who need the current equipment type
            patients_with_need = [p for p in present_patients if len(p.needs) > i and p.needs[i] == 1]
            total_needed = len(patients_with_need) # Count of patients who need the equipment

            # Only enforce hard constraint if availability is greater than or equal to demand
            if total_available >= len(patients_with_need):
                for p in patients_with_need:
                    # Ensure that a patient needing this equipment is assigned to a room that has it
                    assign_with_eq = [seats[(r.id, p.id)] for r in rooms_with_equipment]
                    model.AddBoolOr(assign_with_eq)


        # Soft Constraint Violations - Lists to store boolean variables for each type of violation
        viol_pref_cap = [] # Violations of preferred room capacity
        viol_gender = [] # Violations of gender preference
        viol_pref_eq = [] # Violations of preferred equipment
        viol_need_eq = [] # Violations of needed equipment (when not a hard constraint)
        viol_age = [] # Violations of department age limits
        viol_spec_minor = [] # Minor specialism mismatches (penalty 2)
        viol_spec_major = [] # Major specialism mismatches (penalty >= 3)
        viol_spec_mismatch = [] # Specialism not supported by the room
        viol_fairness_high = [] # Fairness violations above the high threshold
        viol_fairness_medium = [] # Fairness violations between low and high thresholds
        viol_fairness_low = [] # Fairness violations below the low threshold
        move_penalties = [] # Penalties for moving a patient to a different room


        # Populate the violation lists based on potential assignments
        for p in present_patients:
            prev_room = previous_assignments.get(p.id) # Get the patient's previous room assignment
            for r in rooms:
                s = seats[(r.id, p.id)] # The boolean variable for this patient-room assignment

                # Check for each type of soft constraint violation if this assignment is made
                if r.capacity > p.prefered_capacity:
                    viol_pref_cap.append(s)
                if r.gender in ('M','F') and r.gender != p.gender:
                    viol_gender.append(s)
                # Use min length to avoid IndexError if equipment lists have different lengths
                equip_len = min(len(p.prefers), len(r.has))
                if any(p.prefers[i] == 1 and r.has[i] == 0 for i in range(equip_len)):
                    viol_pref_eq.append(s)

                equip_need_len = min(len(p.needs), len(r.has))
                if any(p.needs[i] == 1 and r.has[i] == 0 for i in range(equip_need_len)):
                    viol_need_eq.append(s)
                if (r.department==1 and p.age<65) or (r.department==4 and p.age>16): # Specific age limits for departments 1 and 4
                    viol_age.append(s)

                # Check for specialism compatibility and penalty level
                if p.spec_id in r.spec_ids:
                    idx = r.spec_ids.index(p.spec_id) # Find the index of the patient's specialism in the room's specialism list
                    pen = r.penalties[idx] # Get the penalty value associated with that specialism in the room
                    if pen == 2: # Minor specialism mismatch (penalty 2)
                        viol_spec_minor.append(s)
                    elif pen >= 3: # Major specialism mismatch (penalty >= 3)
                        viol_spec_major.append(s)
                else: # Specialism not supported by the room
                    viol_spec_mismatch.append(s)

                # Calculate fairness score with historical penalty bias
                base_pen = compatibility_penalty(p, r, weights) # Calculate base penalty using the compatibility_penalty function with weights
                if current_day > 0:
                    bias = 1 + historical_penalties[p.id] / max(1, current_day + 1) # Apply bias based on historical penalty
                    fairness_score = base_pen * bias
                else:
                    fairness_score = base_pen # For the first day, fairness score is the base penalty

                # Check for fairness violations based on calculated thresholds
                if fairness_score > threshold_high:
                    viol_fairness_high.append(s)
                elif fairness_score > threshold_low:
                    viol_fairness_medium.append(s)
                elif fairness_score < threshold_low:
                    viol_fairness_low.append(s)
                # --- Move Penalty --- Add penalty if the patient is moved to a different room
                if prev_room is not None and r.id != prev_room:
                    move_penalties.append(s)


        # Total number of patient-seat pairs (for normalization) - Used to scale the objective function
        normalizer = len(present_patients) * len(rooms)

        # Safe normalization function for OR-Tools - Helper function for scaling sums of boolean variables
        def normalized_sum(viol_list):
            return sum(viol_list) * (1.0 / normalizer)

        # Minimize scaled and weighted sum - Define the objective function to minimize total weighted violations
        model.Minimize(
              Wcap * normalized_sum(viol_pref_cap) # Weighted sum of preferred capacity violations
            + Wgen * normalized_sum(viol_gender) # Weighted sum of gender violations
            + Wpeq * normalized_sum(viol_pref_eq) # Weighted sum of preferred equipment violations
            + Wneq * normalized_sum(viol_need_eq) # Weighted sum of needed equipment violations
            + Wage * normalized_sum(viol_age) # Weighted sum of age violations
            + Wspec_minor * normalized_sum(viol_spec_minor) # Weighted sum of minor specialism violations
            + Wspec_major * normalized_sum(viol_spec_major) # Weighted sum of major specialism violations
            + Wspec_mismatch * normalized_sum(viol_spec_mismatch) # Weighted sum of specialism mismatch violations
            + Wfair_low * normalized_sum(viol_fairness_low) # Weighted sum of low fairness violations
            + Wfair_med * normalized_sum(viol_fairness_medium) # Weighted sum of medium fairness violations
            + Wfair_high * normalized_sum(viol_fairness_high) # Weighted sum of high fairness violations
            + Wmove * normalized_sum(move_penalties) # Weighted sum of move penalties
        )

        # 4) Solve the model for the current day
        solver = cp_model.CpSolver() # Create a CP-SAT solver instance
        start = time.time() # Record the start time for solving
        status = solver.Solve(model) # Solve the defined model
        day_duration = time.time() - start # Calculate the time taken to solve the day
        total_duration += day_duration # Add the day's duration to the total duration

        print(f"Day {current_day} solved in {day_duration:.2f}s — {solver.StatusName(status)}") # Print solving status and time

        # 5) Process results if a solution is found
        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            assign = {} # Dictionary to store the assigned room for each patient
            violation_scores = [] # List to store detailed violation scores for each patient-assignment
            daily_score = 0 # Total penalty score for the current day

            # Iterate through present patients to find their assigned room and calculate penalties
            for p in present_patients:
                assigned_room = None
                for r in rooms:
                    if solver.Value(seats[(r.id, p.id)]): # Check if the boolean variable for this assignment is true
                        assigned_room = r # Assign the room
                        assign[p.id] = r.id # Store the assignment
                        previous_assignments[p.id] = r.id # Update the previous assignment for this patient
                        break # Move to the next patient once the room is found

                # If a room was assigned, calculate its penalty and update historical penalties
                if assigned_room:
                    pen = compatibility_penalty(p, assigned_room, weights) # Calculate the compatibility penalty with weights
                    daily_score += pen # Add to the daily total score
                    historical_penalties[p.id] += pen  # Update cumulative historical penalties

                    # Example equipment labels (update this list to match your dataset's actual columns and order)
                    equipment_labels = ['telemetry', 'oxygen', 'nitrogen', 'television']  # Ensure this matches the actual dataset

                    # Only consider the minimum length to avoid IndexError in cases of inconsistent data
                    max_equipment_index = min(len(p.needs), len(assigned_room.has), len(equipment_labels))

                    # Find missing equipment indices
                    missing_eq = [
                        i for i in range(max_equipment_index)
                        if p.needs[i] == 1 and assigned_room.has[i] == 0
                    ]

                    # Determine effective lengths for equipment lists for violation checks
                    equip_len = min(len(p.prefers), len(assigned_room.has))
                    need_len = min(len(p.needs), len(assigned_room.has))
                    # Specialism violation categorization
                    spec_minor = 0
                    spec_major = 0
                    spec_mismatch = 0

                    # Specialism 1
                    if p.spec_id in assigned_room.spec_ids:
                        idx = assigned_room.spec_ids.index(p.spec_id)
                        spec_pen = assigned_room.penalties[idx]
                        if spec_pen == 2:
                            spec_minor += 1
                        elif spec_pen >= 3:
                            spec_major += 1
                    else:
                        spec_mismatch += 1

                    # Append detailed violation scores for the current patient's assignment
                    violation_scores.append({
                        'patient_id': p.id,
                        'room_id': assigned_room.id,
                        'total_score': pen,
                        'historical_penalty': historical_penalties[p.id],
                        'violations': {
                            'capacity': int(assigned_room.capacity > p.prefered_capacity), # 1 if preferred capacity violated, 0 otherwise
                            'gender': int(assigned_room.gender in ['F', 'M'] and assigned_room.gender != p.gender), # 1 if gender mismatch, 0 otherwise
                            'equipment_pref': int(any(p.prefers[i] == 1 and assigned_room.has[i] == 0 for i in range(equip_len))), # 1 if preferred equipment is missing, 0 otherwise
                            'equipment_need': int(any(p.needs[i] == 1 and assigned_room.has[i] == 0 for i in range(need_len))), # 1 if needed equipment is missing (and not a hard constraint violation), 0 otherwise
                            'age': int((assigned_room.department == 1 and p.age < 65) or (assigned_room.department == 4 and p.age > 16)),  # 1 if age limit violated, 0 otherwise (Update if using dynamic department age limits)
                            'specialism': int(spec_minor > 0 or spec_major > 0 or spec_mismatch > 0), # 1 if any specialism violation occurred, 0 otherwise
                            'moved': int(previous_assignments.get(p.id) is not None and assigned_room.id != previous_assignments.get(p.id)), # 1 if patient was moved, 0 otherwise
                        }
                    })

                # Save results
                all_assignments[current_day] = {
                    'assignments': assign, # Assigned room for each patient
                    'violation_scores': violation_scores # Detailed violation scores
                }

                # Violation Counters (dynamic for equipment length)
                move_count = sum(v['violations']['moved'] for v in violation_scores) # Count of patients who were moved


            # Store the assignments and violation scores for the current day
            all_assignments[current_day] = {
                'assignments': assign,
                'violation_scores': violation_scores
            }

            # Append summary results for the current day to the results list
            results.append({
                'day': current_day, # Current day
                'num_patients': len(present_patients), # Number of patients present
                'preferred_capacity_violations': sum(v['violations']['capacity'] for v in violation_scores), # Total preferred capacity violations
                'gender_violations': sum(v['violations']['gender'] for v in violation_scores), # Total gender violations
                'pref_equip_violations': sum(v['violations']['equipment_pref'] for v in violation_scores), # Total preferred equipment violations
                'need_equip_violations': sum(v['violations']['equipment_need'] for v in violation_scores), # Total needed equipment violations
                'age_violations': sum(v['violations']['age'] for v in violation_scores), # Total age violations
                'specialism_violations': sum(v['violations']['specialism'] for v in violation_scores), # Total specialism violations
                'move_violations': sum(v['violations']['moved'] for v in violation_scores), # Total move violations
                'fairness_violations': sum(1 for v in violation_scores if v['total_score']  > threshold_high), # Total fairness violations (total score > high threshold)
                'total_violation_score': daily_score, # Total daily penalty score
                'threshold_low': threshold_low, # Low fairness threshold for the day
                'threshold_high': threshold_high # High fairness threshold for the day
            })

    # 6) Final metrics and reporting - Calculate overall statistics and generate reports
    if results: # Proceed only if there are results
        df = pd.DataFrame(results) # Convert results list to a pandas DataFrame
        total_days = len(all_days) # Total number of days in the planning horizon
        success_rate = len(results) / total_days * 100 # Percentage of days successfully solved
        patient_ids = list(historical_penalties.keys()) # List of all patient IDs
        # Calculate the length of stay for each patient
        stay_length = {p.id: max(1, p.release_day - p.admission_day) for p in patients}
        # Calculate the average daily penalty for each patient
        avg_daily_penalty = {pid: historical_penalties[pid]/stay_length[pid] for pid in stay_length}
        penalty_vals = list(avg_daily_penalty.values()) # List of average daily penalty values

        # Calculate fairness metrics (Standard Deviation, Gini, Jain)
        std_dev = np.std(penalty_vals)
        gini = compute_gini(penalty_vals)
        jain = compute_jain(penalty_vals)

        # Calculate average penalties for all patients and find min/max
        avg_penalties = [historical_penalties[pid]/stay_length[pid] for pid in patient_ids]
        max_penalty_idx = np.argmax(penalty_vals) # Index of patient with maximum average daily penalty
        min_penalty_idx = np.argmin(penalty_vals) # Index of patient with minimum average daily penalty
        # Count total penalties for fairness and FCFS constraints
        total_fairness_penalties = df['fairness_violations'].sum() # Calculate the total fairness violations across all days

        print(f"Total Fairness Penalties: {total_fairness_penalties}") # Print total fairness penalties


        # Build summary stats - Create a dictionary for the summary statistics table
        summary_stats = {
            "Metric": [
                "Total Solving Time (s)",
                "Success Rate (%)",
                "Fairness - StdDev of Daily Penalty",
                "Fairness - Gini Coefficient",
                "Fairness - Jain Index",
                "Total Penalty - Mean",
                "Total Penalty - StdDev",
                "Total Penalty - Min",
                "Total Penalty - Max",
                "Avg Daily Penalty - Mean",
                "Avg Daily Penalty - StdDev",
                "Avg Daily Penalty - Min",
                "Avg Daily Penalty - Max",
                "Patients with Zero Penalty",
                "Patients with High Penalty (>mean+std)",
                f"Most Penalized Patient (ID {patient_ids[max_penalty_idx]})",
                f"Least Penalized Patient (ID {patient_ids[min_penalty_idx]})",
            ],
            "Value": [
                f"{total_duration:.2f}", # Formatted total solving time
                f"{success_rate:.2f}", # Formatted success rate
                f"{std_dev:.2f}", # Formatted standard deviation
                f"{gini:.2f}", # Formatted Gini coefficient
                f"{jain:.2f}", # Formatted Jain index
                f"{np.mean(penalty_vals):.2f}", # Formatted mean total penalty
                f"{np.std(penalty_vals):.2f}", # Formatted standard deviation of total penalty
                f"{np.min(penalty_vals):.2f}", # Formatted minimum total penalty
                f"{np.max(penalty_vals):.2f}", # Formatted maximum total penalty
                f"{np.mean(avg_penalties):.2f}", # Formatted mean average daily penalty
                f"{np.std(avg_penalties):.2f}", # Formatted standard deviation of average daily penalty
                f"{np.min(avg_penalties):.2f}", # Formatted minimum average daily penalty
                f"{np.max(avg_penalties):.2f}", # Formatted maximum average daily penalty
                f"{sum(1 for p in penalty_vals if p == 0)}", # Count of patients with zero penalty
                f"{sum(1 for p in penalty_vals if p > np.mean(penalty_vals) + np.std(penalty_vals))}", # Count of patients with high penalty
                f"Total: {penalty_vals[max_penalty_idx]:.2f}, Avg: {avg_penalties[max_penalty_idx]:.2f}", # Details for most penalized patient
                f"Total: {penalty_vals[min_penalty_idx]:.2f}, Avg: {avg_penalties[min_penalty_idx]:.2f}", # Details for least penalized patient
            ]
        }

        # Create a DataFrame for the summary statistics
        summary_df = pd.DataFrame(summary_stats)

        # Print the summary statistics
        print("\n=== SUMMARY STATISTICS ===")
        print(summary_df.to_string(index=False))
        # Save summary statistics as a CSV file
        summary_df.to_csv("hospital_summary_statistics3.csv", index=False)

        # Save as PNG image - Create a table image of the summary statistics
        fig, ax = plt.subplots(figsize=(12, len(summary_df) * 0.4)) # Create figure and axes
        ax.axis('off') # Turn off axes
        table = ax.table(cellText=summary_df.values, # Create table from DataFrame values
                         colLabels=summary_df.columns, # Use DataFrame columns as column labels
                         cellLoc='left', # Align cell text to the left
                         loc='center') # Center the table in the axes
        table.auto_set_font_size(False) # Disable automatic font sizing
        table.set_fontsize(10) # Set font size for table text
        table.scale(1, 1.2) # Scale the table size
        plt.title("Hospital Allocation Summary Statistics", fontsize=14, weight='bold', pad=20) # Add a title
        plt.tight_layout() # Adjust layout to prevent labels overlapping
        plt.savefig("hospital_summary_statistics3.png", dpi=300) # Save the table as a PNG image
        plt.close() # Close the plot

    # Return the results (all assignments, rooms, patients, and daily summary results)
    return all_assignments, rooms, patients, results

In [ ]:
# --- Reorganized Experiments Based on Better Grouping ---
# Experiment A: Wmove, Wfair_high, Wneq (high importance) - Varying weights for movement penalty, high fairness violations, and needed equipment violations
Wmove_vals = [6, 8, 10] # Possible values for Wmove
Wfair_high_vals = [5, 7, 9] # Possible values for Wfair_high
Wneq_vals = [6, 8, 10] # Possible values for Wneq
experiment_A = list(product(Wmove_vals, Wfair_high_vals, Wneq_vals)) # Generate all combinations for Experiment A

# Experiment B: Wspec_mismatch, Wspec_major, Wage (medium importance) - Varying weights for specialism mismatch, major specialism violations, and age violations
Wspec_mismatch_vals = [4, 6, 7] # Possible values for Wspec_mismatch
Wage_vals = [4, 6, 8] # Possible values for Wage (re-used, might be a typo in experiment grouping)
Wspec_major_vals = [4, 6,8] # Possible values for Wspec_major
experiment_B = list(product(Wspec_mismatch_vals, Wage_vals, Wspec_major_vals)) # Generate all combinations for Experiment B

# Experiment C: Wfair_med, Wgen, Wpeq (equipment & preference) - Varying weights for medium fairness violations, gender violations, and preferred equipment violations
Wfair_med_vals = [4, 6, 8] # Possible values for Wfair_med
Wgen_vals = [3, 5, 7] # Possible values for Wgen
Wpeq_vals = [2, 3, 5] # Possible values for Wpeq
experiment_C = list(product(Wfair_med_vals, Wgen_vals, Wpeq_vals)) # Generate all combinations for Experiment C

# Experiment D: Wcap, Wspec_minor, Wfair_low (lower importance/general) - Varying weights for capacity violations, minor specialism violations, and low fairness violations
Wcap_vals = [2, 3, 5] # Possible values for Wcap
Wspec_minor_vals = [1, 2, 3] # Possible values for Wspec_minor
Wfair_low_vals = [0, 1, 2] # Possible values for Wfair_low
experiment_D = list(product(Wcap_vals, Wspec_minor_vals, Wfair_low_vals)) # Generate all combinations for Experiment D


# Function to run and collect results from different experiments
def run_experiment(experiments, fixed_weights, label):
    results = [] # List to store results for each combination
    infeasible_cases = [] # List to store details of infeasible cases
    # Iterate through each combination of weights in the experiment
    for combo in experiments:
        weights = fixed_weights.copy() # Start with the fixed base weights
        # Update the 'vary' weights with values from the current combination
        for i, key in enumerate([k for k in weights if weights[k] == 'vary']):
            weights[key] = combo[i]

        print(f"Running: {weights}") # Print the current weights being used
        result = weights.copy() # Initialize the result dictionary with the current weights
        try:
            # Attempt to solve the hospital allocation problem with the current weights
            _, rooms, patients, daily_results = solve_hospital_allocation(**weights)

            # If successful, update the result dictionary with performance metrics
            result.update({
                'total_cost': sum(r['total_violation_score'] for r in daily_results), # Total violation score across all days
                'fairness_std': sum(r['fairness_violations'] for r in daily_results) / len(daily_results), # Average fairness violations per day
                'gini': sum(r.get('gini', 0) for r in daily_results) / len(daily_results), # Average Gini coefficient per day (if available)
                'jain': sum(r.get('jain', 0) for r in daily_results) / len(daily_results), # Average Jain's index per day (if available)
                'num_patients': len(patients), # Total number of patients
                'total_rooms': len(rooms), # Total number of rooms
                'status': 'feasible' # Indicate that a feasible solution was found
            })
        except Exception as e:
            # If an exception occurs (e.g., infeasible solution), record the weights and error
            print(f"Infeasible or error: {weights} → {e}")
            infeasible_cases.append({'weights': weights, 'error': str(e)})
            # Update the result dictionary to indicate an infeasible case
            result.update({
                'total_cost': None,
                'fairness_std': None,
                'gini': None,
                'jain': None,
                'num_patients': None,
                'total_rooms': None,
                'status': 'infeasible'
            })
        results.append(result) # Add the result for the current combination to the results list

    df = pd.DataFrame(results) # Convert the results list to a pandas DataFrame
    df.to_csv(f"experiment_{label}.csv", index=False) # Save the results DataFrame to a CSV file
    print(f"Saved experiment {label} to experiment_{label}.csv") # Print confirmation of saving

    # If there were any infeasible cases, save them to a JSON file
    if infeasible_cases:
        with open(f"infeasible_{label}.json", "w") as f:
            json.dump(infeasible_cases, f, indent=2) # Save the infeasible cases list as JSON with indentation
        print(f" Saved {len(infeasible_cases)} infeasible configs to infeasible_{label}.json") # Print confirmation of saving infeasible cases

    return df # Return the results DataFrame

# Default values for all weights (base weights for experiments)
base_weights = {
    'Wcap': 0, 'Wgen': 0, 'Wpeq': 0, 'Wneq': 0, 'Wage': 0,
    'Wspec_minor': 0, 'Wspec_major': 0, 'Wspec_mismatch': 0, 'Wfair_high': 0,
    'Wfair_med': 0, 'Wfair_low': 0,'Wmove': 0
}
# Run each experiment
# exp_A = run_experiment(experiment_A, {**base_weights, 'Wmove': 'vary', 'Wfair_high': 'vary', 'Wneq': 'vary'}, label='A')
# exp_B = run_experiment(experiment_B, {**base_weights, 'Wspec_mismatch': 'vary', 'Wage': 'vary', 'Wspec_major': 'vary'}, label='B')
# exp_C = run_experiment(experiment_C, {**base_weights, 'Wfair_med': 'vary', 'Wgen': 'vary', 'Wpeq': 'vary'}, label='C')
exp_D = run_experiment(experiment_D, {**base_weights, 'Wcap': 'vary', 'Wspec_minor': 'vary', 'Wfair_low': 'vary'}, label='D')


from google.colab import files # Import the files module from google.colab for file downloading
files.download("experiment_D.csv") # Download the results CSV file for Experiments

Running: {'Wcap': 2, 'Wgen': 0, 'Wpeq': 0, 'Wneq': 0, 'Wage': 0, 'Wspec_minor': 1, 'Wspec_major': 0, 'Wspec_mismatch': 0, 'Wfair_high': 0, 'Wfair_med': 0, 'Wfair_low': 0, 'Wmove': 0}
Solving day 0...
Day 0 (initial) thresholds - Low: 0.00, High: 2.00
Day 0 solved in 0.32s — OPTIMAL
Solving day 1...
Day 1 thresholds - Low: 0.00, High: 2.00
Day 1 solved in 0.61s — OPTIMAL
Solving day 2...
Day 2 thresholds - Low: 0.00, High: 2.00
Day 2 solved in 1.09s — OPTIMAL
Solving day 3...
Day 3 thresholds - Low: 0.00, High: 2.00
Day 3 solved in 2.73s — OPTIMAL
Solving day 4...
Day 4 thresholds - Low: 0.00, High: 2.00
Day 4 solved in 4.76s — OPTIMAL
Solving day 5...
Day 5 thresholds - Low: 0.00, High: 2.00
Day 5 solved in 5.84s — OPTIMAL
Solving day 6...
Day 6 thresholds - Low: 0.00, High: 2.00
Day 6 solved in 6.57s — OPTIMAL
Solving day 7...
Day 7 thresholds - Low: 0.00, High: 2.00
Day 7 solved in 8.58s — OPTIMAL
Solving day 8...
Day 8 thresholds - Low: 0.00, High: 2.00
Day 8 solved in 9.50s — OPTIM